# Even: Data Assignment

## Config

Setting up the environment for the analysis.


* This notebook uses `Python3.9` via `miniconda`. The environment can be created with `conda create -n even python=3.9`

In [ ]:
# black formatter
%load_ext nb_black

In [ ]:
import pytz

import datetime as dt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
IST_TZ = pytz.timezone("Asia/Kolkata")
DATETIME_NOW_IST = dt.datetime(year=2023, month=3, day=12, tzinfo=IST_TZ)

In [ ]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)

    percentile_.__name__ = "percentile_%s" % n
    return percentile_

## Data Preparation & Description

In this section we're preparing the data for future analyses. We're trying to also validate whether it's sanity and whether it follows real life trends through some proxy metrics.

In [ ]:
sign_up_info_df = pd.read_csv("data/data_science_task_dataset.csv", index_col=[0])
sign_up_info_df.reset_index(drop=True, inplace=True)
sign_up_info_df.reset_index(names=["id"], inplace=True)

In [ ]:
sign_up_info_df.head()

In [ ]:
# assuming times are in IST
sign_up_info_df["signup_time"] = pd.to_datetime(
    sign_up_info_df["signup_time"]
).dt.tz_localize(IST_TZ)
sign_up_info_df["payment_time"] = pd.to_datetime(
    sign_up_info_df["payment_time"]
).dt.tz_localize(IST_TZ)

sign_up_info_df["plan_months"] = sign_up_info_df["plan_months"].astype(int)
sign_up_info_df["payment_amount"] = sign_up_info_df["payment_amount"].astype(float)
sign_up_info_df["is_early_bird"] = sign_up_info_df["is_early_bird"].astype(bool)

Where ever paytime is missing, payment amount should be `NA`, i.e, it's still pending.

In [ ]:
sign_up_info_df.loc[sign_up_info_df["payment_time"].isna(), "payment_amount"] = np.nan

In [ ]:
sign_up_info_df["signup_to_payment_time"] = (
    sign_up_info_df["payment_time"] - sign_up_info_df["signup_time"]
)

* Each row represents a sign up with multiple family members:
    - represented by multiple values for `genders`, `ages`, `plans`.
    - other rows are common among family members.
* When `payment_time` is null, the customer hasn't paid for the plan yet.

In [ ]:
sign_up_info_df["num_members"] = sign_up_info_df["ages"].apply(
    lambda x: len(x.split(", "))
)

In [ ]:
# splitting each family member to a unique column
sign_up_info_exploded_df = sign_up_info_df.assign(
    **{
        "plans": sign_up_info_df["plans"].str.split(", "),
        "genders": sign_up_info_df["genders"].str.split(", "),
        "ages": sign_up_info_df["ages"].str.split(", "),
    }
).explode(column=["ages", "genders", "plans"])

In [ ]:
sign_up_info_exploded_df["ages"] = sign_up_info_exploded_df["ages"].astype(int)

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6), dpi=120)
sns.countplot(
    x="ages",
    data=sign_up_info_exploded_df,
    order=np.arange(
        start=sign_up_info_exploded_df["ages"].min(),
        stop=sign_up_info_exploded_df["ages"].max(),
        step=1,
    ),
    ax=ax,
)
ax.set_title("Count Plot of ages")
ax.set_xlabel("Age (in years)")
ax.set_ylabel("Count of people")
plt.show()

In [ ]:
sign_up_info_exploded_df["age_bins"] = pd.cut(
    sign_up_info_exploded_df["ages"],
    bins=[0, 17, 35, 50, 65],
    include_lowest=True,
    precision=0,
)
sign_up_info_exploded_df["age_bins"].value_counts(normalize=True, dropna=False) * 100

1. The age limit for insurance of 65 years is line with the [IRDAI](https://www.tataaig.com/knowledge-center/health-insurance/age-limit-for-health-insurance).
2. A major part of the population purchasing premiums at even are between 17-35.

## When do we receive our lowest sign ups? 

And what measures can we take to improve them?

In [ ]:
sign_up_info_df["signup_month"] = sign_up_info_df["signup_time"].dt.month
sign_up_info_exploded_df["signup_month"] = sign_up_info_exploded_df[
    "signup_time"
].dt.month

In [ ]:
(
    sign_up_info_df[sign_up_info_df["payment_time"].notnull()]
    .groupby(["signup_month"])["id"]
    .count()
    / sign_up_info_df.groupby(["signup_month"])["id"].count()
).reset_index().rename(columns={"id": "signup_to_payment_conversion"})

The highest conversion was during month 1, 2, 6, and 7. Why was this the case?

Could January and Feburary have high conversion due to the early bird offer?

In [ ]:
early_bird_sign_up_info_df = sign_up_info_df[sign_up_info_df["is_early_bird"] == True]

In [ ]:
early_bird_sign_up_info_df["payment_time"].isna().sum()

In [ ]:
early_bird_sign_up_info_df["signup_time"].min(), early_bird_sign_up_info_df[
    "signup_time"
].max()

In [ ]:
sign_up_info_df.loc[
    (
        sign_up_info_df["signup_time"]
        >= dt.datetime(year=2022, month=1, day=1, tzinfo=IST_TZ)
    )
    & (
        sign_up_info_df["signup_time"]
        < dt.datetime(year=2022, month=3, day=1, tzinfo=IST_TZ)
    )
]["payment_amount"].isna().sum()

An early bird is anyone who signed up in the month of January or Feburary and has paid for their plan. Is conversion higher here to due to a reduced price?

In [ ]:
sign_up_info_df[sign_up_info_df["payment_time"].notnull()].groupby(["signup_month"])[
    "payment_amount"
].agg(["mean", percentile(25), "median", percentile(75), percentile(90), "count"])

* From the above, it's clear that months one and two paid lesser prices on average compared to other months.
* What's interesting is that even after having higher prices in month 6 and 7, the conversion recovered -- what was that?
* But this could be because the user personas signing up each month are different. 

We can validate this by checking the composition of personas over each month are similar for:
1. Ages
2. Gender
3. Plans

#### Ages

In [ ]:
sign_up_info_exploded_df["age_bins"] = pd.cut(
    sign_up_info_exploded_df["ages"],
    bins=[0, 10, 20, 30, 40, 50, 65],
    include_lowest=True,
    precision=0,
)

age_bins_composition_by_sign_up_month_df = sign_up_info_exploded_df.pivot_table(
    index=["signup_month"],
    columns=["age_bins"],
    values="id",
    aggfunc="count",
    margins=True,
)

age_bins_composition_by_sign_up_month_df.div(
    age_bins_composition_by_sign_up_month_df.iloc[:, -1], axis=0
)

In [ ]:
ages_by_signup_month = sns.FacetGrid(
    data=sign_up_info_exploded_df,
    col="signup_month",
    col_wrap=3,
    height=4,
    sharex="col",
    sharey="row",
)
ages_by_signup_month.map(
    sns.histplot,
    "ages",
    kde=True,
    color="green",
    stat="density",
)
ages_by_signup_month.fig.subplots_adjust(top=0.9)
ages_by_signup_month.fig.suptitle(
    "Histogram of the distribution of payment amount by months"
)
plt.show()

We can see that through the table and the distribution plots above that the ages are distributed similarly. Also fun to note is that the distribution of ages is a bit right skewed with a thicker tail on the right.

#### Gender

In [ ]:
gender_composition_by_sign_up_month_df = sign_up_info_exploded_df.pivot_table(
    index=["signup_month"],
    columns=["genders"],
    values="id",
    aggfunc="count",
    margins=True,
)

gender_composition_by_sign_up_month_df.div(
    gender_composition_by_sign_up_month_df.iloc[:, -1], axis=0
)

All the months follow the same distribution among genders. There's a 50-50 split.

#### Plans

In [ ]:
plan_composition_by_sign_up_month_df = sign_up_info_exploded_df.pivot_table(
    index=["signup_month"],
    columns=["plans"],
    values="id",
    aggfunc="count",
    margins=True,
)

plan_composition_by_sign_up_month_df.div(
    plan_composition_by_sign_up_month_df.iloc[:, -1], axis=0
)

In [ ]:
plan_months_composition_by_sign_up_month_df = sign_up_info_exploded_df.pivot_table(
    index=["signup_month"],
    columns=["plan_months"],
    values="id",
    aggfunc="count",
    margins=True,
)

plan_months_composition_by_sign_up_month_df.div(
    plan_months_composition_by_sign_up_month_df.iloc[:, -1], axis=0
)

There's a similar trend accross plans except for the first two months -- this could possible be because early customers might prefer the cheaper plan.

Whatever differences in conversion could be a result of the price. Let's have a look at this distribution.

In [ ]:
paid_sign_up_info_df = sign_up_info_df.loc[sign_up_info_df["payment_time"].notnull()]

In [ ]:
price_distribution_by_month = sns.FacetGrid(
    data=paid_sign_up_info_df,
    col="signup_month",
    col_wrap=3,
    height=4,
    sharex="col",
    sharey="row",
)
price_distribution_by_month.map(
    sns.histplot,
    "payment_amount",
    kde=True,
    color="green",
    stat="density",
)
price_distribution_by_month.fig.subplots_adjust(top=0.9)
price_distribution_by_month.fig.suptitle(
    "Histogram of the distribution of payment amount by months"
)
plt.show()

In [ ]:
paid_sign_up_info_df.groupby(["signup_month", "plan_months"])["payment_amount"].agg(
    ["mean", percentile(25), "median", percentile(75), percentile(90), "count"]
)

The distribution of plan months is the same AND the prices for each of the plans is significantly increasing. Is there something else that we're doing that's improving our conversion? What's the hidden factor here?

In [ ]:
paid_sign_up_info_df["payment_month"] = paid_sign_up_info_df["payment_time"].dt.month
paid_sign_up_info_df["payment_month"] = paid_sign_up_info_df["payment_time"].dt.month

In [ ]:
paid_sign_up_info_df.groupby(["signup_month"])["signup_to_payment_time"].agg(
    ["mean", percentile(25), "median", percentile(75), percentile(90), "count"]
)

In [ ]:
paid_sign_up_info_df["signup_month_ne_payment_month"] = (
    paid_sign_up_info_df["signup_month"] != paid_sign_up_info_df["payment_month"]
)

In [ ]:
paid_sign_up_info_df.groupby(["signup_month", "signup_month_ne_payment_month"])[
    "id"
].count() 

In [ ]:
(
    paid_sign_up_info_df.groupby(["signup_month", "signup_month_ne_payment_month"])[
        "id"
    ].count()
    / paid_sign_up_info_df.groupby(["signup_month"])["id"].count()
).reset_index().rename(columns={"id": "proportion_by_sign_up_payment_month_mismatch"})

## Can we start a abandoned campaign?

What's the average time to sign up? When can we intervene?

### Corollary

Some additional metrics that we could benefit from during sign ups are:

1. What is the scope for a re-activation campaign?
    - What's our current customer retention? This would help improve customer lifetime value?
    - What's causing them to drop off? What are their expectations from the app? What's their feedback?
2. Where are customers coming from?
    - What's the acquistion source and the conversion of that source? E.g. are they more likely to convert via social media influencer coupons?
    - What do the demographic of those signing up quickly? Is it just older people?
3. Why are customers droppping off?
    - Are people unhappy with the pricing? What does the competitor price and value proposition look like?
    - Are older folks (those above 63) looking for insurance? What's the scope of offering insurance there?
    - Tech issues:
        - Is a payment method broken? – track through conversion by payment method
        - Is the app difficult to use on a certain platform or device type?
        - Are we losing event data on a particular device?

## User Sign Up to Payment Time

Consider the fields `signup_time` and `payment_time`. They stand for the time a given user (who then may add multiple family members) signed up and then paid, respectively. In a single plot, how can you best show the distribution of time "deltas" between the sign up time and payment time (i.e. how long it takes for people to pay once they have signed up)? What is the best way to condense the relevant information and insights? Remember it needs to be a single, static plot, which ideally should not need to be magnified to make sense.

## Mining the Underlying Price

You are given the payment amounts but you don't know what the underlying price function is, and what its inputs are (though you can assume they are a subset of the given fields). If you had to treat this as a prediction problem, what kind of model would you use? **PLEASE DO NOT ACTUALLY ATTEMPT MODELLING**. Base your answer on any data exploration you did (and feel free to show plots/stats), but what we are looking is simply a discussion of what may be some of the modelling challenges here and how to pick a model which can overcome them.

Surviorship bias -- modelling the price the customer is willing to pay.

Linear regression.